<a href="https://colab.research.google.com/github/Komal77rao/Data-Eng-Modules/blob/main/3-adapter-object/index-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The adapter object

### Introduction

In previous lessons, we changed our code to use a receipt class.  The main change was to create a receipt class.

In [2]:
class Receipt:
    def __init__(self, total, address, end_date, restaurant_name):
        self.total = int(total)
        self.address = address
        self.end_date = end_date
        self.restaurant_name = restaurant_name

Now, let's continue to refactor our code.  We can do this by looking for code that's not wrapped in either a method or an object, and that's the code that we still need to clean.

### 1. Identify Messy Code

This is the code we would like to refactor.

In [3]:
import requests
response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
restaurant_receipts = response.json()

receipts = []
for receipt in restaurant_receipts:
    receipt = Receipt(receipt['total_receipts'], receipt['location_address'],
                      receipt['obligation_end_date_yyyymmdd'], receipt['location_name'])
    receipts.append(receipt)

In [4]:
receipts[0].__dict__

{'total': 0,
 'address': '3600 MCKINNEY AVE STE 100',
 'end_date': '2015-08-31T00:00:00.000',
 'restaurant_name': "MAX'S WINE DIVE"}

Now let's turn this into functions.  Ideally any function should not be longer than five lines long, and should not be more than one if else statement (ideally zero if else statements).

Let's go.

### 2. Move messy code into an object

The first step when we have a collection of messy code, is simply to wrap it in an object.

In [5]:
import requests
class ReceiptBuilder:
    def run(self):
        response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
        restaurant_receipts = response.json()
        receipts = []
        for restaurant_receipt in restaurant_receipts:
            receipt = Receipt(restaurant_receipt['total_receipts'],
                              restaurant_receipt['location_address'], restaurant_receipt['obligation_end_date_yyyymmdd'], restaurant_receipt['location_name'])
            receipts.append(receipt)
        return receipts

> So above, we take our code from above and place it inside of our ReceiptBuilder class.  Then, before making any more changes, we check that our code still works.

In [6]:
builder = ReceiptBuilder()
builder.run()[0:2]

[<__main__.Receipt at 0x7be0785d4ca0>, <__main__.Receipt at 0x7be0785d63b0>]

This is pretty nice.  We simply create a new instance of ReceiptBuilder, call our `run` method, and then immediately get a list of receipts.

### 3. Try: Refactor into smaller methods

Now our `run` method is fairly long.  Ideally we want our methods to be no more than five lines long.  A good technique is to write comments above the code and then turn the comments into methods.

Give it a shot in the lines below.  You'll know that you didn't break any thing if you can still call `run` and receive back the same list of receipts.

In [10]:
import requests
class ReceiptBuilder:
    def run(self):
        response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
        restaurant_receipts = response.json()
        return self.receipt(restaurant_receipts)


    def receipt(self, restaurant_receipts):
        receipts = []
        for restaurant_receipt in restaurant_receipts:
            receipt = Receipt(restaurant_receipt['total_receipts'], restaurant_receipt['location_address'],
                              restaurant_receipt['obligation_end_date_yyyymmdd'], restaurant_receipt['location_name'])
            receipts.append(receipt)
        return receipts

In [11]:
builder = ReceiptBuilder()
builder.run()[0:2]

[<__main__.Receipt at 0x7be0781f2020>, <__main__.Receipt at 0x7be0781f2710>]

### Check Answer: Making methods smaller

Ok, let's see how we could have made our `run` method smaller.  Remember that this involves two steps:

1. Write comments above complicated code
2. Turn comments into method names

Let's start by adding some comments.

In [ ]:
import requests
class ReceiptBuilder:
    def run(self):
        # retrieve restaurant receipts
        response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
        restaurant_receipts = response.json()

        # turn receipts into objects
        receipts = []
        for restaurant_receipt in restaurant_receipts:

            receipt = Receipt(restaurant_receipt['total_receipts'], restaurant_receipt['location_address'],
                              restaurant_receipt['obligation_end_date_yyyymmdd'], restaurant_receipt['location_name'])
            receipts.append(receipt)
        return receipts

2. Turn comments into methods.

Now let's change the comments into methods.

In [12]:
import requests
class ReceiptBuilder:
    def retrieve_receipts(self):
        response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
        return response.json()

    def receipts_data_to_objects(self, restaurant_receipts):
        receipts = []
        for restaurant_receipt in restaurant_receipts:
            receipt = Receipt(restaurant_receipt['total_receipts'],
                              restaurant_receipt['location_address'],
                              restaurant_receipt['obligation_end_date_yyyymmdd'], restaurant_receipt['location_name'])
            receipts.append(receipt)
        return receipts

    def run(self):
        self._receipts_data = self.retrieve_receipts()
        self._receipts = self.receipts_data_to_objects(self._receipts_data)
        return self._receipts

In [13]:
builder = ReceiptBuilder()
builder.run()[0:3]

At this point, our `receipts_data_to_objects` method still is too long so once again, the steps are to read the long method, make comments to see the steps involved, and then turn those comments into methods.

### Tidying Up

This is what we have as our final version of the class.

Once again, we need our receipt class.

In [ ]:
from datetime import datetime
class Receipt:
    def __init__(self, total, address, end_date, restaurant_name):
        self.total = int(total)
        self.address = address
        self.end_date = end_date
        self.restaurant_name = restaurant_name

And this is our new `ReceiptBuilder`.

In [ ]:
import requests
class ReceiptBuilder:
    def run(self):
        self._receipts_data = self.retrieve_receipts()
        self._receipts = self.receipts_data_to_objects(self._receipts_data)
        return self._receipts

    def retrieve_receipts(self):
        response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
        return response.json()

    def create_receipt(self, receipt_data):
        receipt = Receipt(receipt_data['total_receipts'], receipt_data['location_address'], receipt_data['obligation_end_date_yyyymmdd'], receipt_data['location_name'])
        return receipt

    def receipts_data_to_objects(self, receipts_data):
        receipts = []
        for receipt_data in receipts_data:
            receipt = self.create_receipt(receipt_data)
            receipts.append(receipt)
        return receipts

In [ ]:
builder = ReceiptBuilder()
builder.run()[0:3]

### Summary

In this lesson we saw how to take a collection of unorganized code and make it more organized.  We did this through a step by step procedure.

1. Identify the messy code
2. Create a new class to place the messy code, and place code in MessyClass.run() method
3. Make methods smaller by
    * Write comments explaining what parts of the method do
    * Turn those comments into individual method calls